# Import Necessary Libraries

In [4]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Total Images for Each Class

In [18]:
training_dir = 'C:\\Users\\JoshyBoi\\WGU C964 Final Capstone\\Dataset\\Training'
validating_dir = 'C:\\Users\\JoshyBoi\\WGU C964 Final Capstone\\Dataset\\Validating'
testing_dir = 'C:\\Users\\JoshyBoi\\WGU C964 Final Capstone\\Dataset\\Testing'

species_images = {}

for species in os.listdir(training_dir):
    species_class = os.path.join(training_dir, species)
    images = len([image for image in os.listdir(species_class)])
    species_images[species] = images

for species, count in species_images.items():
    print(f"{species}: {count} images")

Clam: 2078 images
Crab: 2138 images
Dolphin: 1848 images
Fish: 2154 images
Jellyfish: 2007 images
Lobster: 2198 images
Octopus: 1927 images
Puffer: 2198 images
Seal: 2039 images
Shrimp: 2000 images
Squid: 2145 images
Starfish: 2065 images
Stingray: 1968 images
Turtle: 2143 images
Whale: 2202 images


# Gather File Paths and Labels

In [19]:
training_paths = []
training_labels = []

for label in os.listdir(training_dir):
    species_class = os.path.join(training_dir, label)
    for image in os.listdir(species_class):
        training_paths.append(os.path.join(species_class, image))
        training_labels.append(label)

training_df = pd.DataFrame({'ImagePath': training_paths, 'Label': training_labels})

training_df.head()

,ImagePath,Label
0,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
1,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
2,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
3,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
4,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam


In [20]:
validating_paths = []
validating_labels = []

for label in os.listdir(validating_dir):
    species_class = os.path.join(validating_dir, label)
    
    for image in os.listdir(species_class):
        validating_paths.append(os.path.join(species_class, image))
        validating_labels.append(label)

validating_df = pd.DataFrame({'ImagePath': validating_paths, 'Label': validating_labels})

validating_df.head()

,ImagePath,Label
0,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
1,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
2,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
3,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
4,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam


In [21]:
testing_paths = []
testing_labels = []

for label in os.listdir(testing_dir):
    species_class = os.path.join(testing_dir, label)
    
    for image in os.listdir(species_class):
        testing_paths.append(os.path.join(species_class, image))
        testing_labels.append(label)

testing_df = pd.DataFrame({'ImagePath': testing_paths, 'Label': testing_labels})

testing_df.head()

,ImagePath,Label
0,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
1,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
2,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
3,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam
4,C:\Users\JoshyBoi\WGU C964 Final Capstone\Data...,Clam


# Data Augmentation and Preprocessing

In [8]:
training_dataset = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True)

validation_dataset = ImageDataGenerator(rescale=1./255)
testing_dataset = ImageDataGenerator(rescale=1./255)

training_images = training_dataset.flow_from_directory(
    training_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32)

validation_images = validation_dataset.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32)

testing_images = testing_dataset.flow_from_directory(
    testing_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32)

Found 31073 images belonging to 15 classes.
Found 750 images belonging to 15 classes.
Found 300 images belonging to 15 classes.


# Adding Custom Layers

In [11]:
mobile = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg')

mobile.trainable = False

model = Sequential([
    mobile,
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(15, activation='softmax')])

model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dense_6 (Dense)             (None, 1024)              1311744   
                                                                 
 dropout_4 (Dropout)         (None, 1024)              0         
                                                                 
 dense_7 (Dense)             (None, 512)               524800    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 15)                7695      
                                                      

# Improve Training Process

In [13]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True)

checkpoint = ModelCheckpoint(
    'updated_model.h5',
    monitor='val_accuracy',
    save_best_only=True)

history = model.fit(
    training_images,
    validation_data=validation_images,
    epochs=50,
    callbacks=[early_stop, checkpoint])

Epoch 1/50
972/972 [==============================] - 453s 460ms/step - loss: 2.5119 - accuracy: 0.2187 - val_loss: 1.6498 - val_accuracy: 0.6227
Epoch 2/50
972/972 [==============================] - 463s 476ms/step - loss: 1.7365 - accuracy: 0.4634 - val_loss: 1.1426 - val_accuracy: 0.6973
Epoch 3/50
972/972 [==============================] - 433s 446ms/step - loss: 1.4031 - accuracy: 0.5646 - val_loss: 0.9586 - val_accuracy: 0.7280
Epoch 4/50
972/972 [==============================] - 413s 425ms/step - loss: 1.2478 - accuracy: 0.6122 - val_loss: 0.8668 - val_accuracy: 0.7520
Epoch 5/50
972/972 [==============================] - 429s 442ms/step - loss: 1.1669 - accuracy: 0.6337 - val_loss: 0.8087 - val_accuracy: 0.7667
Epoch 6/50
972/972 [==============================] - 452s 465ms/step - loss: 1.0910 - accuracy: 0.6585 - val_loss: 0.7660 - val_accuracy: 0.7800
Epoch 7/50
972/972 [==============================] - 438s 450ms/step - loss: 1.0396 - accuracy: 0.6724 - val_loss: 0.7400 -